### Warning
Please run the "Fetching Datasets" notebook before and create csv files corresponding to both BeerAdvocate and RateBeer ratings.txt.gz :)

In [ ]:
import pandas as pd
import seaborn as sns
import os 
from functions import read_data
import re


In [ ]:
COLUMNS_NAMES = ["count","beer_name","beer_id","brewery_name","brewery_id","style","abv","date","username","user_id","appearance","aroma","palate","taste","overall","rating","text","review"]

In [ ]:
#Check all the RateBeer csvs are there where they should be
import os 
RB_csv_list = os.listdir("../data")
RB_csv_list.sort()
RB_csv_list = [csv for csv in RB_csv_list if ".csv" in csv and 'ratings' in csv and 'RateBeer' in csv]
print(RB_csv_list)

In [ ]:
#Check all the BeerAdvocate csvs are there where they should be
BA_csv_list = os.listdir("../data")
BA_csv_list.sort()
BA_csv_list = [csv for csv in BA_csv_list if ".csv" in csv and 'ratings' in csv and 'BeerAdvocate' in csv]
print(BA_csv_list)

In [ ]:
def aggregate_ratings(dataframe):
    dataframe.fillna({'count_x':0,'count_y':0,'mean_rating_x':0,'mean_rating_y':0, "min_rating_x" : float('inf'),"max_rating_x": -float('inf'),"min_rating_y" : float('inf'),"max_rating_y": -float('inf')},inplace=True)    
    dataframe["count2"] = dataframe['count_x'] + dataframe['count_y']
    
    dataframe["min_rating"] =  dataframe[['min_rating_x','min_rating_y']].min(axis=1)
    dataframe['mean_rating'] = (dataframe['mean_rating_x']*dataframe['count_x'] + dataframe['mean_rating_y']*dataframe['count_y'])/dataframe["count2"]
    dataframe["max_rating"] =  dataframe[['max_rating_x','max_rating_y']].max(axis=1)
    dataframe["count"] = dataframe["count2"]
    dataframe.drop(columns=["min_rating_x","mean_rating_x","max_rating_x","count_x","min_rating_y","mean_rating_y","max_rating_y","count_y","count2"],inplace=True)
    return dataframe
def multiple_csv(dataframe,dataframe2):
    COLUMNS_NAMES = ["count","beer_name","beer_id","brewery_name","brewery_id","style","abv","date","username","user_id","appearance","aroma","palate","taste","overall","rating","text"]

    # 5 is the length of the aggregated df, if the incoming df has more than 5 columns, it was not aggregated yet.
    if len(dataframe.columns) > 5:
        dataframe.columns = COLUMNS_NAMES
        grouped = dataframe.groupby(by="user_id").agg({'rating': ['min','mean', 'max'], 'count': 'count'}).reset_index(level=[0])
        grouped.columns = ["user_id","min_rating","mean_rating","max_rating","count"]
    else :
        grouped = dataframe
        grouped.columns = ["user_id","min_rating","mean_rating","max_rating","count"]

    dataframe2.columns = COLUMNS_NAMES

    grouped2 = dataframe2.groupby(by="user_id").agg({'rating': ['min','mean', 'max'], 'count': 'count'}).reset_index(level=[0])
    
    grouped2.columns = ["user_id","min_rating","mean_rating","max_rating","count"]
    #print("-------")

    df = aggregate_ratings(pd.merge(grouped, grouped2, how="outer",on=['user_id']))
    #display(pd.merge(grouped, grouped2, how="outer",on=['user_id']))
    #display(df)
    #print("-------")

    return df
    #test1[["appearance" ,"aroma","palate","taste","overall"]]=test1[["appearance" ,"aroma","palate","taste","overall"]].apply(pd.to_numeric)


In [ ]:
#change range to correspond to reality if you have not created the csvs with 500'000 columns.
BA_dataframe = pd.read_csv("../data/BeerAdvocate_ratings_part_0.csv")
for index in range(1,17):
    temp = pd.read_csv(f"../data/BeerAdvocate_ratings_part_{index}.csv")
    print(f"Aggregating csv {index}")
    BA_dataframe = multiple_csv(BA_dataframe,temp)


In [ ]:
RB_dataframe = pd.read_csv("../data/RateBeer_ratings_part_0.csv")
for index in range(1,15):
    temp = pd.read_csv(f"../data/RateBeer_ratings_part_{index}.csv")
    print(f"Aggregating csv {index}")
    RB_dataframe = multiple_csv(RB_dataframe,temp)

In [ ]:
RB_dataframe

In [ ]:
# Fetch csv with user data so we can get Country of origin:
BA_users_df = read_data.fetch_csv("../DATA/BeerAdvocate.tar","users")
BA_users_df = BA_users_df[BA_users_df["nbr_ratings"] >= 1].copy()
RB_users_df = read_data.fetch_csv("../DATA/RateBeer.tar","users")
RB_users_df = RB_users_df[RB_users_df["nbr_ratings"] >= 1].copy()

In [ ]:
# Sanity check : are the histograms of the dataframe I created and the given dataset equal?
sns.histplot(data =RB_dataframe, x = "count",log_scale=True)

In [ ]:
# Answer : Yes ! Change to BA to check other site, everything should be ok!
sns.histplot(data =RB_users_df, x = "nbr_ratings",log_scale=True)

In [ ]:
BA_users_df.user_id=BA_users_df.user_id.str.strip()
BA_dataframe.user_id=BA_dataframe.user_id.str.strip()
BA_combined_df = pd.merge(BA_users_df, BA_dataframe, how="left",on=['user_id'])
BA_combined_df["user_id"] = BA_combined_df["user_id"].map(lambda x : re.search(r"[^.]+$",x)[0])
BA_combined_df["location"].fillna("Empty",inplace=True)
#mean aggregation in the next line is wrong, redo it
BA_countries = BA_combined_df.groupby('location').agg({'nbr_ratings': 'sum','min_rating':'min','mean_rating':'mean','max_rating':'max','count':'sum'})
display(BA_countries)


In [ ]:
RB_combined_df = pd.merge(RB_users_df, RB_dataframe, how="left",on=['user_id'])
RB_combined_df["location"].fillna("Empty",inplace=True)
#mean aggregation in the next line is wrong, redo it
RB_countries = RB_combined_df.groupby('location').agg({'nbr_ratings': 'sum','min_rating':'min','mean_rating':'mean','max_rating':'max','count':'sum'})
display(RB_countries)
